In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.special import logit, expit
from scipy.stats import uniform, norm, bernoulli, mannwhitneyu
from matplotlib import pyplot as plt
import pymc as pm
import arviz as az
from modeltools import mcmc_diagnostics, create_summary_stat
from downcast import downcast_df
from simulatescores import simulate_scores
import jax
from pymc.sampling_jax import sample_numpyro_nuts

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
/home/roger/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/roger/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/roger/.local/lib/python3.10/site-packages/pymc/sampling_jax.py:37: UserWarning: This module is experimental.
  warnings.warn("This module is experimental.")


In [2]:
#  Setting up seeds
seed = 42

# Setting numpy seed
np.random.seed(seed)

# GPU setting
GPU = True

In [3]:
jax.devices()

[StreamExecutorGpuDevice(id=0, process_index=0)]

In [4]:
import sys
import importlib
importlib.reload(sys.modules['simulatescores'])
from simulatescores import simulate_scores

In [5]:
raw_data = pd.read_csv("data/unit_level_ratings.csv",index_col = 0)
raw_data = raw_data.sort_values(by=["corpus", "model", "topic"])

In [6]:
# Creating identifier for each corpus, model, and topic
# Identifier is unique for topic 
corpus_ids = (raw_data.groupby(["corpus"], as_index=False)
    .agg({"intrusion":"count"})
    .drop(columns="intrusion"))
corpus_ids["corpus_id"] = corpus_ids.index

model_ids = (raw_data.groupby(["model"], as_index=False)
    .agg({"intrusion":"count"})
    .drop(columns="intrusion"))
model_ids["model_id"] = model_ids.index

cordel_ids = (raw_data.groupby(["corpus", "model"], as_index=False)
    .agg({"intrusion":"count"})
    .drop(columns="intrusion"))
cordel_ids["cordel_id"] = cordel_ids.index 

topic_ids = (raw_data.groupby(["corpus", "model", "topic"], as_index=False)
    .agg({"intrusion":"count"})
    .drop(columns="intrusion"))
topic_ids["topic_id"] = topic_ids["topic"].astype(np.int16)

rater_ids = (raw_data.groupby(["corpus", "rater"], as_index=False)
    .agg({"intrusion":"count"})
    .drop(columns="intrusion"))
rater_ids["rater_id"] = rater_ids.index 


d1 = pd.merge(raw_data, corpus_ids, on=["corpus"], how="left")
d2 = pd.merge(d1, model_ids, on=["model"], how="left")
d3 = pd.merge(d2, cordel_ids, on=["corpus","model"], how="left")
d4 = pd.merge(d3, rater_ids, on=["corpus", "rater"], how="left")
data = pd.merge(d4, topic_ids, on=["corpus", "model", "topic"], how="left")
data = data[["corpus_id", "model_id", "cordel_id", "topic_id", "rater_id", "intrusion", "confidence"]]
data, na_s = downcast_df(data)

In [7]:
# Setting up numpy arrays for pymc
corpus_array = np.array(data["corpus_id"])
n_corpora = data["corpus_id"].nunique()

model_array = np.array(data["model_id"])
n_models = data["model_id"].nunique()

cordel_array = np.array(data["cordel_id"])
n_cordels = data["cordel_id"].nunique()

topic_array = np.array([data["cordel_id"], data["topic_id"]])
n_topics = data["topic_id"].nunique()

rater_array = np.array(data["rater_id"])
n_raters = data["rater_id"].nunique()

score_array = np.array(data["intrusion"])

## Bayesian Model

In [8]:
# Model and MCMC specifications

n_cores = 2
empirical_mean = logit(0.75)
r_lambda = 2
t_lambda = 1
t_sigma = 1
# cm_lambda = 2
# cm_sigma = 1
mu_sigma = 1

glm_rater_topic_cordel = {"model":pm.Model()}

In [13]:
# Rater, Topic, Cordel model

glm_rater_topic_cordel["model"] = pm.Model()
with glm_rater_topic_cordel["model"]:
    # Hyperparameter priors
    raters = pm.Data("raters", rater_array, mutable=True, dims="obs_id")
    topics = pm.Data("topics", topic_array, mutable=True, dims=["cordel", "topic"])
    cordels = pm.Data("cordels", cordel_array, mutable=True, dims="obs_id")
    
    sigma_r = pm.Exponential("sigma_r", lam=r_lambda)
    zr = pm.Normal("zr",mu=0, sigma=1, shape=n_raters)
    sigma_a = pm.Exponential("sigma_a", lam=t_lambda)
    za = pm.Normal("za",mu=0, sigma=t_sigma, shape=(n_cordels, n_topics)) 
    mu = pm.Normal("mu",mu=empirical_mean, sigma=mu_sigma, shape=n_cordels)
    
    s = pm.Bernoulli(
            "s", 
            p=pm.math.invlogit(
                mu[cordels]+
                za[topics[0],topics[1]]*sigma_a+
                zr[raters]*sigma_r),
            observed=score_array, 
            dims="obs_id")

    c_mean = pm.Deterministic("c_mean", 
                              pm.math.invlogit(mu + (za.T*sigma_a)).mean(axis=0), 
                              dims="obs_id")
    
    if GPU:
        glm_rater_topic_cordel["trace"]=sample_numpyro_nuts(random_seed=seed)#, chain_method="vectorized")
    else:
        glm_rater_topic_cordel["trace"]=pm.sample(cores=n_cores, random_seed=seed)

Compiling...


/home/roger/.local/lib/python3.10/site-packages/pymc/sampling_jax.py:548: UserWarning: There are not enough devices to run parallel chains: expected 4 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(4)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  pmap_numpyro = MCMC(


Compilation time =  0:00:01.366462
Sampling...


sample: 100%|█| 2000/2000 [00:41<00:00, 47.96it/s, 15 steps of size 2.12e-01. ac
sample: 100%|█| 2000/2000 [00:56<00:00, 35.67it/s, 63 steps of size 1.92e-01. ac
sample: 100%|█| 2000/2000 [00:54<00:00, 36.62it/s, 31 steps of size 1.88e-01. ac
sample: 100%|█| 2000/2000 [00:58<00:00, 34.19it/s, 63 steps of size 1.93e-01. ac


Sampling time =  0:03:31.396917
Transforming variables...
Transformation time =  0:00:00.057619
Computing Log Likelihood...
Log Likelihood time =  0:00:00.196473


In [ ]:
glm_rater_topic_cordel["summary_stat"] = create_summary_stat(glm_rater_topic_cordel["trace"])

## Functions

In [ ]:
def calculate_raters(n_raters=None, scores_per_rater=None, total_scores=None):
    count_none = sum([n_raters ==None, scores_per_rater ==None, total_scores ==None])
    assert(count_none == 2)
    
    if n_rater == None:
        n_raters = total_scores//scores_per_rater + 1
    
    if scores_per_rater == None:
        scores_per_rater = total_scores//n_raters + 1
    
    if total_scores == None:
        total_scores = n_raters*scores_per_rater
    
    return n_raters, scores_per_rater, total_scores
         

In [ ]:
def utest_pval(scores):
    # Utests whether the two distributions in the scores are statisticaly significant
    # returns 1 if cordel 1 intrusion scores is statistically higher
    # returns 0 if no statistical difference
    # returns -1 if cordel 2 intrusion scores are statistically higher 

    sums = scores.groupby(["sim_cordel_id", "sim_topic_id"]).agg({"intrusion":"sum"}).reset_index()
    cordel0_intrusions = sums[sums["sim_cordel_id"]==0]["intrusion"]
    cordel1_intrusions = sums[sums["sim_cordel_id"]==1]["intrusion"]
    p_val = mannwhitneyu(cordel0_intrusions, cordel1_intrusions, alternative="two-sided")[1]
    
    return p_val


In [ ]:
def bht_pval(sample):
# sample = scores[scores["sim_id"]==0]

# Bayesian hypothesis tests whether the two distributions in the sample are statisticaly significant
# Setting up numpy arrays for pymc
# Only 2 models and 1 corpus in simulation
    corpus_array = np.array([0]*len(sample))
    n_corpora = 1

    model_array = np.array(sample["sim_cordel_id"])
    n_models = sample["sim_cordel_id"].nunique()

    cordel_array = np.array(sample["sim_cordel_id"])
    n_cordels = sample["sim_cordel_id"].nunique()

    topic_array = np.array([sample["sim_cordel_id"], sample["sim_topic_id"]])
    n_topics = sample["sim_topic_id"].nunique()

    rater_array = np.array(sample["sim_rater_id"])
    n_raters = sample["sim_rater_id"].nunique()

    score_array = np.array(sample["intrusion"])


    # Model and MCMC specifications
    n_cores = 2
    empirical_mean = logit(0.75)
    r_lambda = 2
    t_lambda = 1
    t_sigma = 1
    # cm_lambda = 2
    # cm_sigma = 1
    mu_sigma = 1

    glm = {"model":pm.Model()}

    # Rater, Topic, Cordel model

    glm["model"] = pm.Model()
    with glm["model"]:
        # Hyperparameter priors
        raters = pm.Data("raters", rater_array, mutable=True, dims="obs_id")
        topics = pm.Data("topics", topic_array, mutable=True, dims=["cordel", "topic"])
        cordels = pm.Data("cordels", cordel_array, mutable=True, dims="obs_id")

        sigma_r = pm.Exponential("sigma_r", lam=r_lambda)
        zr = pm.Normal("zr",mu=0, sigma=1, shape=n_raters)
        sigma_a = pm.Exponential("sigma_a", lam=t_lambda)
        za = pm.Normal("za",mu=0, sigma=t_sigma, shape=(n_cordels, n_topics)) 
        mu = pm.Normal("mu",mu=empirical_mean, sigma=mu_sigma, shape=n_cordels)

        s = pm.Bernoulli(
                "s", 
                p=pm.math.invlogit(
                    mu[cordels]+
                    za[topics[0],topics[1]]*sigma_a+
                    zr[raters]*sigma_r),
                observed=score_array, 
                dims="obs_id")

        c_mean = pm.Deterministic("c_mean", 
                                  pm.math.invlogit(mu + (za.T*sigma_a)).mean(axis=0), 
                                  dims="obs_id")
        c_diff = pm.Deterministic("c_diff", c_mean.reshape([n_cordels,1]) - c_mean.reshape([1,n_cordels]), dims="obs_id")

        glm["trace"]=pm.sample(cores=n_cores, progressbar=False)

    n_negatives = (glm["trace"].posterior["c_diff"].sel({"obs_id":1, "c_diff_dim_1":0}) < 0).sum().item()
    
    return  n_negatives/len(sample)

In [ ]:
def generate_sim_settings(n_trials, p_diff, n_raters=None, scores_per_r=None, total_scores=None):
#     n_trials = 10
#     p_diff = (0.02, 0.2)
#     n_raters = (20, 100)
#     scores_per_r = 40
#     total_scores = 1300
    # Checking only 2 of 3 score and rater variables is declared
    count_none = sum([n_raters == None, scores_per_r == None, total_scores == None])
    assert count_none == 1, "There should be 2 score/rater variables declared"

    # Sampling uniform random variables
    if type(p_diff) == tuple:
        col_p_diff = uniform.rvs(loc = p_diff[0], scale=p_diff[1]-p_diff[0], size=n_trials)
    else:
        col_p_diff = np.array([p_diff]*n_trials)

    if type(n_raters) == tuple:
        col_n_raters = np.random.randint(n_raters[0], high=n_raters[1], size=n_trials)
    else:
        col_n_raters = np.array([n_raters]*n_trials)

    if type(scores_per_r) == tuple:
        col_scores_per_r = np.random.randint(scores_per_r[0], high=scores_per_r[1], size=n_trials)
    else:
        col_scores_per_r = np.array([scores_per_r]*n_trials)
    
    if type(total_scores) == tuple:
        col_total_scores = np.random.randint(total_scores[0], high=total_scores[1], size=n_trials)
    else:
        col_total_scores = np.array([total_scores]*n_trials)

    # Calculating remaining column
    if n_raters == None:
        col_n_raters = (col_total_scores-1)//col_scores_per_r + 1
    elif scores_per_r == None:
        col_scores_per_r = (col_total_scores-1)//col_n_raters + 1
    elif total_scores == None:
        col_total_scores = col_scores_per_r * col_n_raters
    else:
        raise Exception("How did you even get this exception? Should've been impossible, but congratulations")
        
    df = pd.DataFrame(
        np.array([col_p_diff, col_n_raters, col_scores_per_r, col_total_scores]).T,
        columns=["p_diff", "n_raters", "scores_per_r", "total_scores"])
    
    df = df.astype({
        "p_diff":float,
        "n_raters":np.uint16,
        "scores_per_r":np.uint16,
        "total_scores":np.uint16
    })

      
    return df

In [ ]:
def simulate_sig_tests(n_trials, p_diff, n_raters=None, scores_per_r=None, total_scores=None, sims_per_trial=1):
#     p_diff = (0.02, 0.2)
#     n_raters = (20, 100)
#     scores_per_r = 40
#     sims_per_trial = 2
#     n_trials = 5

    # Checking only 2 of 3 score and rater variables is declared
    count_none = sum([n_raters == None, scores_per_r == None, total_scores == None])
    assert(count_none == 1)



    sim_results = pd.DataFrame(
        columns=["sim_id", "p_diff", "n_raters", "scores_per_r", "total_scores", "utest_pval", "bht_pval"]
    )
    sim_results = sim_results.astype({
        "p_diff":float,
        "n_raters":np.uint16,
        "scores_per_r":np.uint16, 
        "total_scores":np.uint16,
        "sim_id":int,
        "utest_pval":float, 
        "bht_pval":float,
    })

    settings_df = generate_sim_settings(n_trials=n_trials, p_diff=p_diff, n_raters=n_raters, 
                                        scores_per_r=scores_per_r, total_scores=total_scores)

    for i in range(n_trials):
        p_diff = settings_df.loc[i, "p_diff"].item()
        n_raters = settings_df.loc[i, "n_raters"].item()
        scores_per_r = settings_df.loc[i, "scores_per_r"].item()

        scores = simulate_scores(
            glm_rater_topic_cordel,
            p_diff=p_diff,
            n_raters=n_raters,
            scores_per_r=scores_per_r,
            n_sims=sims_per_trial)

        for j in range(sims_per_trial):
            sim_results = pd.concat([sim_results, pd.DataFrame(
                [[p_diff, n_raters, scores_per_r, j,
                 utest_pval(scores[scores["sim_id"]==j]),
                 bht_pval(scores[scores["sim_id"]==j])]],
                columns=["p_diff", "n_raters", "scores_per_r", "sim_id", "utest_pval", "bht_pval"]
            )], ignore_index=True)

    return sim_results

## Simulation

In [ ]:
# Settings for hoyle's significance testing
hoyle_total_scores = 50*26
hoyle_p_diff = 0.055
hoyle_n_raters = 38
hoyle_scores_per_r = hoyle_total_scores//hoyle_n_raters + 1

In [ ]:
# total ratings between 400 and 2400
# Hoyle used 1300 total ratings
sim_n_raters = simulate_sig_tests(
    p_diff = hoyle_p_diff,
    n_raters = (10, 60),
    scores_per_r = hoyle_scores_per_r,
    sims_per_trial = 1,
    n_trials = 10
)

In [ ]:
sim_scores_per_r = simulate_sig_tests(
    p_diff = hoyle_p_diff,
    scores_per_r = (10, 100),
    total_scores = hoyle_total_scores,
    sims_per_trial = 1,
    n_trials = 10
)

In [ ]:
# total ratings between 400 and 2400
# Hoyle used 1300 total ratings
sim_p_n_raters = simulate_sig_tests(
    p_diff = (0.02, 0.2),
    n_raters = (10, 60),
    scores_per_r = hoyle_scores_per_r,
    sims_per_trial = 1,
    n_trials = 10
)

In [ ]:
sim_p_scores_per_r = simulate_sig_tests(
    p_diff = (0.02, 0.2),
    scores_per_r = (10, 100),
    total_scores = hoyle_total_scores,
    sims_per_trial = 1,
    n_trials = 10
)

## Plots

In [ ]:
sim_results["utest"] = sim_results["utest_pval"] < alpha
sim_results["bht"] = sim_results["bht_pval"] < alpha

# (utest, bht)
# Neither significant: orange
# bht significant: yellow
# utest significant: blue
# both significant: green

sim_results["c"] = np.select(
    condlist=[
        ~sim_results["utest"] & ~sim_results["bht"],
        ~sim_results["utest"] & sim_results["bht"],
        sim_results["utest"] & ~sim_results["bht"],
        sim_results["utest"] & sim_results["bht"]
    ],
    choicelist=["orangered", "yellow", "blue", "green"],
    default="black")


In [ ]:
n_raters_results

In [ ]:
plt.scatter(n_raters_results["p_diff"], n_raters_results["n_raters"], c=n_raters_results["c"])
plt.show()

. \
. \
. \
. \
. \
. \
. \
. 
## Checking score simulation script

In [ ]:
# Simulating scores
p_diff = 0.08
n_raters = 40
scores_per_r = 40
n_sims = 1_000
np.random.seed(seed)

In [ ]:
scores = simulate_scores(
    glm_rater_topic_cordel,
    p_diff=p_diff,
    n_raters=n_raters,
    scores_per_r=scores_per_r)

In [ ]:
sim_scores = scores.copy(deep=True)

In [ ]:
# Checking scores

_, ax = plt.subplots(2,1,figsize=(10,10))

n_ratings_per_cordel = n_raters*scores_per_r/2

sns.kdeplot(sim_scores[sim_scores["sim_cordel_id"]==0].groupby(["sim_id"]).agg({"intrusion":"sum"})["intrusion"]/n_ratings_per_cordel, ax=ax[0])
sns.kdeplot(sim_scores[sim_scores["sim_cordel_id"]==1].groupby(["sim_id"]).agg({"intrusion":"sum"})["intrusion"]/n_ratings_per_cordel, ax=ax[0])
ax[0].legend(ax[0].get_lines(), ["Cordel 0 means", "Cordel 1 means"])
ax[0].set_xlabel("Cordel means (probability)")
ax[0].set_title(f"Simulated means by cordel")

diff = sim_scores[sim_scores["sim_cordel_id"]==1].groupby(["sim_id"]).agg({"intrusion":"sum"}) \
        - sim_scores[sim_scores["sim_cordel_id"]==0].groupby(["sim_id"]).agg({"intrusion":"sum"})

sns.kdeplot(diff["intrusion"]/n_ratings_per_cordel, ax=ax[1])
ax[1].set_title(f"Difference in cordel means")
plt.plot()

print(f"Perc of simulations with mean1 < mean0 (n raters={n_raters}): {(diff['intrusion']<=0).sum()/1000:.1%}")

In [ ]:
size=100

_, ax = plt.subplots(1,1,figsize=(10,5))


# Selecting 100 random simulations
rand_ids = np.random.choice(range(1000), size=size, replace=False)

# Calculating topic proportions
samples = (sim_scores[sim_scores["sim_id"].isin(rand_ids)].groupby(["sim_id", "sim_topic_id"])
           .agg({"intrusion":["sum", "count"]}).reset_index())
samples["p"] = samples[("intrusion", "sum")]/samples[("intrusion", "count")]
# Plotting proportions
for sim_id in rand_ids:
    sns.kdeplot(samples[samples["sim_id"]==sim_id]["p"], ax=ax, color="navy", alpha=7/size**0.9)

# Repeat calculations and plots for actual data
actuals = (data.groupby(["cordel_id", "topic_id"])
           .agg({"intrusion":["sum", "count"]}).reset_index())
actuals["p"]=actuals[("intrusion", "sum")]/actuals[("intrusion", "count")]
for cordel_id in range(6):
    sns.kdeplot(actuals[actuals["cordel_id"]==cordel_id]["p"], ax=ax, color="orange", alpha=0.7)

ax.set_title("Simulated topic probabilities (blue) vs actual topic probabilities (orange)")
plt.show()

In [ ]:
size=100

_, ax = plt.subplots(1,1,figsize=(10,5))


# Selecting 100 random simulations
rand_ids = np.random.choice(range(1000), size=size, replace=False)

# Calculating rater proportions
samples = (sim_scores[sim_scores["sim_id"].isin(rand_ids)].groupby(["sim_id", "sim_rater_id"])
           .agg({"intrusion":["sum", "count"]}).reset_index())
samples["p"] = samples[("intrusion", "sum")]/samples[("intrusion", "count")]
# Plotting proportions
for sim_id in rand_ids:
    sns.kdeplot(samples[samples["sim_id"]==sim_id]["p"], ax=ax, color="navy", alpha=0.05)

# Repeat calculations and plots for actual data
actuals = (data.groupby(["corpus_id", "rater_id"])
           .agg({"intrusion":["sum", "count"]}).reset_index())
actuals["p"]=actuals[("intrusion", "sum")]/actuals[("intrusion", "count")]
for corpus_id in range(2):
    sns.kdeplot(actuals[actuals["corpus_id"]==corpus_id]["p"], ax=ax, color="orange", alpha=1)

ax.set_title("Simulated rater probabilities (blue) vs actual rater probabilities (orange)")
plt.show()